In [1]:
import numpy as np
import pandas as pd
import math
import os

In [2]:
path = "/Users/joyumin/PycharmProjects/hadoop/ConvertResult"

In [3]:
def getSpaceIndex(x, y):
    x_idx = int((num_floor(x, 3) - 126.764) / 0.0035)
    y_idx = int((num_floor(y, 3) - 37.429) / 0.00227)

    return x_idx + (119 * y_idx)

def num_floor(num, f=0):
    f = 10 ** f
    temp = math.floor(num * f);
    return temp / f

In [4]:
dirs = [d for d in os.listdir(path) if os.path.isdir(path + "/" + d)]

In [5]:
def combineIntoPD(d):
    filenames = [f for f in os.listdir(path + "/" + d)]
    arr = []
    print(len(dirs))
    for f in filenames:
        #기본 필터 (서울 특별시, 컬럼, NaN)
        temp_df = pd.read_csv(path + "/" + d + "/" + f, encoding="euc-kr").dropna(axis=0);
        temp_df.columns = temp_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '_').str.replace(')', '')
        temp_df.query("승차위치_x좌표 >  126.764278 and 승차위치_x좌표 < 127.183551 and 승차위치_y좌표 > 37.429010 and 승차위치_y좌표 < 37.701079").filter(["승차시각", "승차위치_x좌표", "승차위치_y좌표"])
        temp_df['승차시각'] = pd.to_datetime(temp_df['승차시각'])
        
        #시간 인덱스 추가
        s = (temp_df.승차시각 - temp_df.승차시각.dt.floor('D')).dt.total_seconds() / 300
        s = s.astype(int)
        temp_df['time_label'] = s
        
        #공간 인덱스 추가
        loc_df = temp_df
        temp_df["loc_idx"] = loc_df.apply(lambda row: getSpaceIndex(row.승차위치_x좌표, row.승차위치_y좌표), axis=1)
        
        #count추가
        temp_df['count'] = temp_df.groupby(['time_label', 'loc_idx'])['loc_idx'].transform('count')
        
        #count_lange 추가
        temp_df['count_lange'] = temp_df['count'].apply(lambda x: 0 if x > 0 and x <= 2 else 1 if x > 2 and x <= 5 else 2 if x > 5 and x <= 10 else 3 if x > 10 and x <= 20 else 4)
        
        arr.append(temp_df)
    result = pd.concat(arr)
    return result;
        

0    false
1     True
2     True
3    false
4    false
Name: 0, dtype: object

In [6]:
result_csv = pd.concat([combineIntoPD(d) for d in dirs])
result_csv.info()

8
8
8
8
8
8
8
8
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61666244 entries, 0 to 641951
Data columns (total 21 columns):
수집일시           object
주행거리_m         int64
영업거리_m         int64
공차거리_m         int64
영업시간_초         int64
금액             int64
승차시각           datetime64[ns]
승차위치_x좌표       float64
승차위치_y좌표       float64
승차위치_시도        object
승차위치_시군구       object
승차위치_읍면동       object
하차시각           object
하차위치_x좌표       float64
하차위치_y좌표       float64
하차위치_시도        object
하차위치_시군구       object
time_label     int64
loc_idx        int64
count          int64
count_lange    int64
dtypes: datetime64[ns](1), float64(4), int64(9), object(7)
memory usage: 10.1+ GB


In [3]:
getSpaceIndex(127.027412, 37.497952) # 강남역

3526

In [9]:
a['count_lange'].value_counts()

1    32006
0    18257
2    13963
3      453
Name: count_lange, dtype: int64

In [7]:
result_csv.to_csv("/Users/joyumin/PycharmProjects/datas/combine2.csv", mode='w', index=False, encoding="euc-kr", chunksize=1000000)